# Assignment 2: Graph
Name: Phillis Ye  
StudentID: 452535825  
UPI: zye615

In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix

## Part 1: Implement the power iteration in matrix form without considering the dead-ends and spider traps.

#### Preprocessing the data

In [2]:
# read the data in.
web_data = pd.read_csv("web-Google.txt", sep=" ", header=None, names=['from', 'to'])

In [3]:
# convert data into numpy array format
web_array = web_data.to_numpy()
print(web_array)

[[     0      1]
 [     0      2]
 [     0      3]
 ...
 [875712  13656]
 [875712   9430]
 [875712 542451]]


In [4]:
# size of the data
web_array.shape

(5105039, 2)

In [5]:
# count the number of outlinks for each node.
unique, counts = np.unique(web_array[:,0], return_counts=True)
outlinks = dict(zip(unique, counts))

### Calculate the rank score for all the nodes and report:

#### (a) The running time and the number of iterations needed to stop.

In [6]:
# The function to create the sparse adjacency matrix M.
def adjacency_matrix(links, outlinks, n):
    # create empty row, col, and value to store the values
    row = list()
    col = list()
    value = list()
    
    # number of lines in the file.
    nrow = len(links)
    # iterate each line in the file to store the directed edge from one node to the other.
    for i in range(nrow):
        # get the from node number and to node number
        from_node = links[i,0]
        to_node = links[i,1]
        
        # find the number of outlinks for the from node.
        out = outlinks[from_node]
        
        # append the value into the row, col, and value, such that value Mji = 1/di
        row.append(to_node)
        col.append(from_node)
        value.append(1/out)
    
    # convert the row, col, and value into numpy array
    row = np.array(row)
    col = np.array(col)
    value = np.array(value)
    
    # construct the compressed sparse row matrix using csr_matrix((data, (row, col)), shape)
    # the matrix is NxN where N is number of nodes in the file.
    matrix = csr_matrix((value, (row, col)), shape = (n,n))
    
    return matrix

# implement the power iteration in matrix form without considering the dead-ends and spider traps.
def power_iteration(M, r, threshold):
    # initialise the iteration number to be zero.
    iteration = 0
    
    # initialise the difference between new_r and old_r to be infinite.
    difference = float("inf")
    # initialise the old_r.
    old_r = r
    
    # the iteration stops while difference < threshold.
    while difference >= threshold:
        # number of iteration adds 1.
        iteration += 1
        
        # new_r equals the matrix M dot product the old_r
        new_r = M.dot(old_r)
        # the difference is the sum of the absolute values of the difference between new_r and old_r.
        difference = np.sum(np.abs(np.subtract(new_r, old_r)))
        # the new_r becomes old_r for next iteration.
        old_r = new_r
    
    # return the final number of iterations and final rank score.
    return iteration, old_r

In [7]:
# number of nodes = 875712+1 because the ID ranging from 0 to 875712.
n = 875712+1

# create the sparse matrix M to store the information of out-links for each node.
sparse_M = adjacency_matrix(web_array, outlinks, n)

print("The sparse matrix M looks like:")
print(sparse_M)

The sparse matrix M looks like:
  (0, 1)	0.07142857142857142
  (0, 2)	0.09090909090909091
  (0, 3)	0.08333333333333333
  (0, 4)	0.1
  (0, 6)	0.041666666666666664
  (0, 8)	0.14285714285714285
  (0, 9)	0.0625
  (0, 12)	0.2
  (0, 13)	0.1
  (0, 14)	0.05555555555555555
  (0, 15)	0.25
  (0, 16)	0.0625
  (0, 19)	0.1111111111111111
  (0, 20)	0.0625
  (0, 21)	0.05555555555555555
  (0, 22)	0.034482758620689655
  (0, 23)	0.0625
  (0, 24)	0.03333333333333333
  (0, 25)	0.07692307692307693
  (0, 26)	0.05263157894736842
  (0, 27)	0.025
  (0, 28)	0.06666666666666667
  (0, 30)	0.2
  (0, 31)	0.2
  (0, 32)	0.16666666666666666
  :	:
  (875585, 875581)	0.1111111111111111
  (875586, 875581)	0.1111111111111111
  (875587, 875581)	0.1111111111111111
  (875588, 875581)	0.1111111111111111
  (875589, 875581)	0.1111111111111111
  (875590, 875581)	0.1111111111111111
  (875592, 875591)	0.2
  (875593, 875591)	0.2
  (875594, 875591)	0.2
  (875595, 875591)	0.2
  (875596, 875591)	0.2
  (875599, 875598)	0.25
  (875603, 8

In [8]:
# initialize the rank score r_0 = [1/N, ..., 1/N]^T
r = np.full((n, 1), 1/n)

print("The running time of implementing the power iteration in matrix form without considering the dead-ends and spider traps is:")

The running time of implementing the power iteration in matrix form without considering the dead-ends and spider traps is:


In [9]:
%timeit power_iteration(sparse_M, r, 0.02)

1.3 s ± 57.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
iteration, rank = power_iteration(sparse_M, r, 0.02)
print("The number of iterations needed to stop is:", str(iteration) + ".")

#### (b) The IDs and scores of the top-10 ranked nodes.

In [ ]:
# the format of the final result
def output_format(node_list):
    data = np.empty((10,3), dtype=object)
    # find the top 10 nodes with highest rank.
    for i in range(10):
        # rank starts at 1 so need to add 1 to the index i.
        rank = int(i+1)
        # find the node IDs with rank i+1
        node_id = int(node_list[i][0])
        # find the scores of the nodes
        score = round(node_list[i][1],7)
        # store the data in a list
        data[i]=[rank,node_id,score]

    # create a data frame to store all the data for top 10 nodes.
    result = pd.DataFrame(data, columns = ['Rank','Node_ID','Score'])
    return result

In [ ]:
# create an ID index array from 0 to 875712 to represent the ID.
index = np.arange(n, dtype=int)
index = np.transpose([index])
# combine the nodes ID and their rank scores.
rank_score = np.append(index,rank,axis=1)

# order the nodes from highest rank score to lowest rank score.
rank_score = rank_score.tolist() 
ordered_rank = sorted(rank_score, key=lambda x: x[1], reverse=True)

print("The IDs and scores for the top-10 ranked nodes are:")
display(output_format(ordered_rank))

## Part 2: Extend the PageRank code to handle spider traps and dead-ends.

In [ ]:
# the updated power iteration function to handle spider traps and dead-ends.
def update_power_iteration(M, r, beta, threshold):
    # initialise the iteration number to be zero.
    iteration = 0
    
    # initialise the difference between new_r and old_r to be infinite.
    difference = float("inf")
    # initialise the old_r.
    old_r = r
    
    # the iteration stops while difference < threshold.
    while difference >= threshold:
        # number of iteration adds 1.
        iteration += 1
        
        # calculate new rank score for each nodes that have outlinks.
        new_r = beta*M.dot(old_r)
        # calculate sum of new rj
        s = np.sum(new_r)
        # update new rj with (1-s)/N
        constant = (1-s)/n
        new_r += constant
        # the difference is the sum of the absolute values of the difference between new_r and old_r.
        difference = np.sum(np.abs(np.subtract(new_r, old_r)))
        # the new_r becomes old_r for next iteration.
        old_r = new_r
    
    # return the final number of iterations and final rank score.
    return iteration, old_r

### Run the code on the Google web data and report:

#### (a) The running time and the number of iterations needed to stop.

In [ ]:
# let beta = 0.9 by default.
beta = 0.9

print("The running time of implementing the power iteration in matrix form with considering the dead-ends and spider traps is:")

In [ ]:
%timeit update_power_iteration(sparse_M, r, beta, 0.02)

In [ ]:
iteration, new_rank = update_power_iteration(sparse_M, r, beta, 0.02)
print("The number of iterations needed to stop is:", str(iteration) + ".")

#### (b) The IDs and scores of the top-10 ranked nodes.

In [ ]:
# combine the IDs with their rank scores.
new_rank_score = np.append(index,new_rank,axis=1)

# order the nodes from highest rank score to lowest rank score.
new_rank_score = new_rank_score.tolist() 
ordered_rank = sorted(new_rank_score, key=lambda x: x[1], reverse=True)

print("The IDs and scores for the top-10 ranked nodes are:")
display(output_format(ordered_rank))

#### (c) By varing the teleport probability $\beta$ in [1, 0.9, 0.8, 0.7, 0.6], report the number of iterations needed to stop for each $beta$, and explain the findings.

In [ ]:
# varying the teleport probabilities.
betas = [1, 0.9, 0.8, 0.7, 0.6]

# for each beta, run the code and report the number of iterations needed to stop.
for beta in betas:
    iteration, new_rank = update_power_iteration(sparse_M, r, beta, 0.02)
    print("The number of iterations needed to stop for beta =", beta, "is:", str(iteration) + ".")

The code above runs the power iteration for each teleport probability $\beta$ in [1, 0.9, 0.8, 0.7, 0.6]. From the output, it shows that the smaller the beta, the less number of iterations needed to stop. When the beta $\beta = 1$, it means we only adjust the graph and link the dead ends to all other nodes with equal probability and compute the power iteration without any probability to randomly jump to another node. Thus, it needs the highest number of iterations to stop. However, with smaller beta, the larger the $(1-\beta)$ which means the higher probability to randomly jump to another node and makes the $(1-\beta ) [\frac{1}{N}]_{NxN}$ larger and thus the rank scores are faster to converge. 

## Part 3: Implement the biased PageRank algorithm.

#### (a) Explain how you address the dead-end problem in biased PageRank.

To address the dead-end problem in biased PageRank, I used the similar approach in Question 2. By definition, in biased PageRank, teleport only to nodes within a set S. The matrix formulation of biased PageRank is
$$r = \beta Mr + (1 - \beta )\frac{1}{|S|} e_S$$
To address the dead-ends, the matrix $M$ needs to transit to a new matrix $M'$ where the dead ends are linked to nodes within the set S with equal probability. The euqation becomes
$$r = \beta M'r + (1 - \beta )\frac{1}{|S|} e_S$$
$$= (\beta Mr + \beta \frac{1}{|S|} e_S r_{deadend}) + (1-\beta )\frac{1}{|S|} e_S$$
$$= \beta Mr + \beta \frac{1}{|S|} e_S r_{deadend} + (1-\beta )\frac{1}{|S|} e_S$$
$$r_j = \beta \sum_{i \rightarrow j} \frac{r_i}{d_i} + \beta \frac{1}{|S|} e_S r_{deadend} + (1-\beta )\frac{1}{|S|} e_S$$
$$= \beta \sum_{i \rightarrow j} \frac{r_i}{d_i} + constant* e_S$$
Therefore, firstly I calculate the new value of each node which is the same as before calculating $\beta Mr$:
$$ r_j^{new} = \beta \sum_{i \rightarrow j} \frac{r_i}{d_i}$$
Then calculate the sum of new $r_j$, the sum is less than 1 because dead-ends leaked the scores:
$$ SUM = \sum_{j} r_j^{new} < 1$$
The difference between 1 and SUM is the leaked scores, it should be equal to the constant times the number of nodes that dead-ends we supposed to linked to, that's the number of nodes in the teleport set.
$$ 1-SUM = C*|S|$$
By rearranging the equation, we get the equation for the constant.
$$ C = \frac{1-SUM}{|S|}$$
Finally, update $r_j^{new}$ with constant C
$$ r_j^{new} = r_j^{new} + \frac{1-SUM}{|S|} e_S$$
And I get the equation above is the method I use to address the dead-end problem in biased PageRank.

### Run the biased PageRank algorithm and report:

#### (b) The running time and the number of iterations needed to stop.

In [ ]:
def e_vector(set):
    # initialise a vector with length n with value 0.
    e = np.zeros(n)
    vector = np.transpose([e])
    # if node i is in the set S, the i-th value equals 1.
    for node in set:
        vector[node] = 1
    return vector

def biased_pagerank(M, r, beta, vector, threshold):
    # initialise the iteration number to be zero.
    iteration = 0
    
    # initialise the difference between new_r and old_r to be infinite.
    difference = float("inf")
    # initialise the old_r.
    old_r = r
    
    # the iteration stops while difference < threshold.
    while difference >= threshold:
        # number of iteration adds 1.
        iteration += 1
        
        # calculate new rank score for each nodes that have outlinks.
        new_r = beta*M.dot(old_r)
        # calculate sum of new rj
        s = np.sum(new_r)
        # update new rj with (1-s)/|S|
        constant = (1-s)/len(teleport_set)
        new_r += constant*vector
        # the difference is the sum of the absolute values of the difference between new_r and old_r.
        difference = np.sum(np.abs(np.subtract(new_r, old_r)))
        # the new_r becomes old_r for next iteration.
        old_r = new_r
    
    # return the final number of iterations and final rank score.
    return iteration, old_r

In [ ]:
# the given teleport set of node IDs
teleport_set = [768277, 296506, 77624, 596601, 234218, 222596, 1376, 783154]
# construct the vector with the i-th value equals 1 if node i is in the teleport set S, otherwise 0.
vector = e_vector(teleport_set)

beta=0.9

print("The running time of implementing the biased PageRank algorithm with considering the dead-ends is:")

In [ ]:
%timeit biased_pagerank(sparse_M, r, beta, vector, 0.02)

In [ ]:
iteration, biased_rank = biased_pagerank(sparse_M, r, beta, vector, 0.02)
print("The number of iterations needed to stop for the biased PageRank algorithm is:", str(iteration) + ".")

#### (c) The IDs and scores of the top-10 tanked nodes.

In [ ]:
# combine the IDs with their rank scores.
biased_rank_score = np.append(index,biased_rank,axis=1)

# order the nodes from highest rank score to lowest rank score.
biased_rank_score = biased_rank_score.tolist() 
ordered_rank = sorted(biased_rank_score, key=lambda x: x[1], reverse=True)

print("The IDs and scores for the top-10 ranked nodes are:")
display(output_format(ordered_rank))